In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, train_test_split

import hervpd as hp

In [2]:
df = pd.read_csv( 'data/12features/combined.csv')
print(df.activity.unique())
df.describe()

['eat' 'focused-active' 'focused-passive' 'household-chores'
 'leisure-active' 'leisure-passive' 'movement' 'sleep' 'rest-active']


,user,AVGNN,SDNN,MeanHR,MinHR,MaxHR,RMSSD,pNNxx,TINN,powerVLF,powerLF,powerHF,ratioHFLF
count,544.000000,544.000000,544.000000,544.00000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000
mean,1.187500,758.600000,72.865018,81.37714,67.362033,108.560417,46.409227,20.082227,358.020221,2877.387599,1423.919671,1000.824001,2.753764
std,0.913249,129.785003,32.960406,13.75457,10.849680,36.385572,32.537407,18.316263,146.885899,3619.576847,1443.217366,1841.656665,2.296974
min,0.000000,475.730000,17.833000,51.62800,41.437000,58.117000,4.882500,0.000000,83.000000,85.654000,57.852000,7.977700,0.154680
25%,0.000000,670.362500,51.871500,73.23225,60.292750,91.470250,27.438750,6.593075,258.000000,995.182500,591.307500,258.917500,1.257000
50%,2.000000,750.545000,67.057000,79.94200,66.181500,100.520000,37.994500,15.417500,339.000000,1807.950000,1040.350000,512.090000,2.037350
75%,2.000000,819.307500,85.820000,89.50400,74.285250,112.580000,54.762250,26.790500,426.250000,3304.250000,1697.950000,960.045000,3.579975
max,2.000000,1162.200000,264.250000,126.12000,104.900000,550.320000,240.470000,98.232000,1103.000000,29716.000000,12958.000000,16103.000000,18.228000


In [ ]:
df.groupby('activity').count()['user']

### Apply scale to make all features go to mean = 0 and SD = 1. That will allow SVM kernels not to put too much weight on features with larger values.

In [ ]:
#df.iloc[:, 3:15] = df.iloc[:, 3:15].apply(lambda x: scale(x))
df = hp.scaleWithinUser(df)
df.describe()

### Test dataset will have 20% of examples, the other 80% will be for training (with cross-validation)

In [ ]:
train, test = train_test_split(df, test_size=0.2)
print (len(train), len(test))

### 3 - Model selection:  Perform an exhaustive search, with k-fold cross-validation,  in the space parameter consisting of:
* $C = 10^{i}, com -1 \leq i \leq 6$, for both linear and RBF kernels
* $\gamma = 10^{i}, com -3 \leq i \leq 3$, for RBF kernel.

In [ ]:
crossval = StratifiedShuffleSplit(n_splits=4, test_size=0.2)

c_range = np.logspace(-1, 2, 4) 
gamma_range = np.logspace(-2, 1, 4)

param_lin=dict(C=c_range)
param_rbf=dict(C=c_range, gamma=gamma_range)


In [ ]:
grid_lin = GridSearchCV(svm.SVC(kernel='linear', cache_size=1000), param_grid=param_lin, cv=crossval)
grid_lin.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel linear --- ")
print("Best params: %s with score %0.5f" % (grid_lin.best_params_, grid_lin.best_score_))

In [ ]:
grid_rbf = GridSearchCV(svm.SVC(kernel='rbf', cache_size=1000), param_grid=param_rbf, cv=crossval)
grid_rbf.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel RBF --- ")
print("Best params: %s with score %0.5f" % (grid_rbf.best_params_, grid_rbf.best_score_))

In [ ]:
clf1 = svm.SVC(kernel='linear', cache_size=1000, C=grid_lin.best_params_['C'])
clf1.fit(X=train.iloc[:, 3:15], y=train['activity'])
hp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))

clf2 = svm.SVC(kernel='rbf', cache_size=1000, C=grid_rbf.best_params_['C'], gamma=grid_rbf.best_params_['gamma'])
clf2.fit(X=train.iloc[:, 3:15], y=train['activity'])
hp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))


## Now, we can run the same steps above with multiple experiments! Time to have fun!!

In [40]:
# exclude from df activities with too few examples

df = pd.read_csv( 'data/12features/combined.csv')
df = hp.excludeActivities(df, ['leisure-active',  'rest-active'])
hp.runFlow(df)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


49 out of 103 right! :)
                  precision    recall  f1-score   support

             eat       0.12      0.29      0.17         7
  focused-active       0.63      0.63      0.63        19
 focused-passive       0.25      0.13      0.17        15
household-chores       0.38      0.31      0.34        16
 leisure-passive       0.53      0.59      0.56        17
        movement       0.54      0.47      0.50        15
           sleep       0.73      0.79      0.76        14

     avg / total       0.49      0.48      0.47       103



### 2 - Removing activities from the list

In [41]:
df_reduced = hp.excludeActivities(df, ['leisure-active',  'rest-active', 'sleep', 'movement', 'household-chores', 'eat'])
df_reduced.groupby('activity').count()['user']

activity
focused-active     88
focused-passive    73
leisure-passive    89
Name: user, dtype: int64

In [42]:
hp.runFlow(df_reduced)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


18 out of 50 right! :)
                 precision    recall  f1-score   support

 focused-active       0.33      0.18      0.24        22
focused-passive       0.41      0.50      0.45        14
leisure-passive       0.33      0.50      0.40        14

    avg / total       0.36      0.36      0.34        50



### 1 - All activities per user

In [32]:
ju = hp.userRows(df, 0)
ron = hp.userRows(df, 1)
edu = hp.userRows(df, 2)

ju.describe()

,user,AVGNN,SDNN,MeanHR,MinHR,MaxHR,RMSSD,pNNxx,TINN,powerVLF,powerLF,powerHF,ratioHFLF
count,171.0,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000
mean,0.0,705.760117,78.435041,87.234982,70.866170,129.230216,64.659298,21.486519,408.152047,2947.679649,1539.862433,1732.321988,1.932863
std,0.0,115.589830,42.775841,14.019577,11.197753,54.078833,44.097554,19.449443,185.021049,4435.316039,2005.189948,2836.653377,1.748349
min,0.0,478.320000,23.498000,51.628000,41.437000,71.208000,14.743000,0.238660,116.000000,155.130000,84.136000,29.782000,0.154680
25%,0.0,618.915000,48.248000,77.871000,64.014000,99.404500,28.788500,5.416700,275.000000,696.530000,473.085000,258.845000,0.716570
50%,0.0,707.750000,70.417000,84.776000,72.150000,111.190000,54.254000,16.935000,397.000000,1601.100000,841.630000,600.950000,1.373800
75%,0.0,770.500000,94.793500,96.943500,78.523500,144.050000,88.970000,32.690000,505.000000,3245.750000,1695.800000,2033.550000,2.809450
max,0.0,1162.200000,264.250000,125.440000,92.937000,550.320000,240.470000,79.351000,1103.000000,29716.000000,12958.000000,16103.000000,11.995000


In [37]:
hp.runFlow(ju)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


16 out of 35 right! :)
                  precision    recall  f1-score   support

             eat       0.00      0.00      0.00         1
  focused-active       0.33      0.50      0.40         2
 focused-passive       0.43      0.75      0.55         8
household-chores       0.00      0.00      0.00         4
 leisure-passive       0.40      0.33      0.36         6
        movement       0.60      0.38      0.46         8
           sleep       0.80      0.67      0.73         6

     avg / total       0.46      0.46      0.44        35



C:\Users\julia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [38]:
hp.runFlow(ron)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


10 out of 15 right! :)
                 precision    recall  f1-score   support

            eat       0.50      0.33      0.40         3
 focused-active       0.56      1.00      0.71         5
leisure-passive       1.00      0.50      0.67         6
       movement       1.00      1.00      1.00         1

    avg / total       0.75      0.67      0.65        15



In [39]:
hp.runFlow(edu)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


26 out of 54 right! :)
                  precision    recall  f1-score   support

             eat       0.25      0.43      0.32         7
  focused-active       0.50      0.86      0.63         7
 focused-passive       0.25      0.50      0.33         4
household-chores       0.50      0.25      0.33        12
 leisure-passive       1.00      0.10      0.18        10
        movement       0.50      0.50      0.50         6
           sleep       0.89      1.00      0.94         8

     avg / total       0.60      0.48      0.45        54



### 2 - Splitting database into movement categories (movement, household chores and exercise) and all others

In [29]:
includelist = ['movement', 'household-chores']

df_move = hp.addPartition(df, includelist, pname='move', labelIn='move', labelOut='still')
df_move.groupby('move').count()['user']

move
move     133
still    378
Name: user, dtype: int64

In [30]:
hp.runFlow(df_move, labelName='move')

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


408 train examples and 103 test examples
Best params for linear kernel: {'C': 1.0} with score 0.90549
Best params for RBF kernel: {'C': 1.0, 'gamma': 0.10000000000000001} with score 0.90244
--- test results for linear kernel:
93 out of 103 right! :)
--- test results for RBF kernel:
93 out of 103 right! :)


In [ ]:
jumove = hp.addPartition(ju, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(jumove, labelName='move')

In [ ]:
edumove = hp.addPartition(edu, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(edumove, labelName='move')

In [ ]:
ronmove = hp.addPartition(ron, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(ronmove, labelName='move')

In [ ]:
includelist
dfnomove = hp.excludeActivities(df, includelist)
dfnomove.describe()
hp.runFlow(dfnomove)

In [ ]:
[x for x in df['activity']]

In [47]:
hp.runFlowByUser(df)

CLASSFIER FOR USER 0
136 train examples and 35 test examples


C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Best params for linear kernel: {'C': 1.0} with score 0.56250
--- test results for linear kernel:
                  precision    recall  f1-score   support

  focused-active       0.00      0.00      0.00         3
 focused-passive       0.43      0.50      0.46         6
household-chores       0.57      0.57      0.57         7
 leisure-passive       1.00      0.29      0.44         7
        movement       0.67      0.50      0.57         8
           sleep       0.33      1.00      0.50         4

     avg / total       0.58      0.49      0.47        35

Best params for RBF kernel: {'C': 10.0, 'gamma': 0.10000000000000001} with score 0.61607
--- test results for RBF kernel:
                  precision    recall  f1-score   support

             eat       0.00      0.00      0.00         0
  focused-active       0.25      0.33      0.29         3
 focused-passive       0.50      0.67      0.57         6
household-chores       0.80      0.57      0.67         7
 leisure-passive       

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Best params for RBF kernel: {'C': 10.0, 'gamma': 0.01} with score 0.75000
--- test results for RBF kernel:
                 precision    recall  f1-score   support

            eat       0.50      0.50      0.50         2
 focused-active       0.71      0.83      0.77         6
leisure-passive       0.40      0.40      0.40         5
       movement       1.00      0.50      0.67         2

    avg / total       0.62      0.60      0.60        15

CLASSFIER FOR USER 2
214 train examples and 54 test examples


C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Best params for linear kernel: {'C': 1.0} with score 0.49419
--- test results for linear kernel:
                  precision    recall  f1-score   support

             eat       0.11      0.33      0.17         3
  focused-active       0.47      0.78      0.58         9
 focused-passive       0.29      0.25      0.27         8
household-chores       0.50      0.40      0.44        10
 leisure-passive       0.50      0.25      0.33         8
        movement       1.00      0.29      0.44         7
           sleep       0.89      0.89      0.89         9

     avg / total       0.57      0.48      0.48        54

Best params for RBF kernel: {'C': 10.0, 'gamma': 0.01} with score 0.41279
--- test results for RBF kernel:
                  precision    recall  f1-score   support

             eat       0.09      0.33      0.14         3
  focused-active       0.43      0.67      0.52         9
 focused-passive       0.33      0.25      0.29         8
household-chores       0.50      0.40 